In [106]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.naive_bayes import GaussianNB
import math
import copy

In [107]:
df=pd.read_csv("LoanDataset/data.csv")
df.columns = ['id','age','exp','income','zip','family','spend','education','house','output','security','certi','net','creditcard']

In [108]:
X = df.drop(['output'],axis=1)
Y = df['output']

In [109]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2)

In [110]:
df = pd.concat([X_train,Y_train],axis=1)

In [111]:
distinct = {}
def seperateClass():
    for index,row in df.iterrows():
        if row['output'] not in distinct:
            distinct[row['output']]=[]
        distinct[row['output']].append(list(row))
# print distinct

In [112]:
def mean(attrColumn):
    return float(sum(attrColumn))/len(attrColumn)

In [113]:
def standard_deviation(attrColumn):
    avg = mean(attrColumn)
    ans = 0
    for i in attrColumn:
        ans=ans+pow(i-avg,2)
    ans = float(ans)/(len(attrColumn)-1)
    return np.sqrt(ans)

In [114]:
def find_mean_sd_for_attr(df):

    summaries = [(mean(attribute), standard_deviation(attribute)) for attribute in zip(*df)]
    del summaries[-1]
    return summaries

In [115]:
seperateClass()
summary = {}
for label,data in distinct.iteritems():
    summary[label] = find_mean_sd_for_attr(data)

In [116]:
def calcProbability(x,mean,sd):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(sd,2))))
    return (1 / (math.sqrt(2*math.pi) * sd)) * exponent

In [117]:
def probability_by_class(inputdata,summary):
    
    probabilities = {}
    for label, labeldata in summary.iteritems():
        probabilities[label] = 1
        for i in range(len(labeldata)):
            mean, sd = labeldata[i]
            x = inputdata[i]
            probabilities[label] *= calcProbability(x, mean, sd)
    return probabilities

In [118]:
def prediction(inputdata,summary):
    probabilities = probability_by_class(inputdata,summary)
    anslabel = None
    maxprob = 0
    for label, probability in probabilities.iteritems():
        if anslabel is None or probability > maxprob:
            maxprob = probability
            anslabel = label
    return anslabel

In [119]:
res = []
for index,row in X_test.iterrows():
    res.append(prediction(row,summary))

print confusion_matrix(Y_test,res)
print classification_report(Y_test,res)
print accuracy_score(Y_test, res)*100

[[727  92]
 [ 12  69]]
              precision    recall  f1-score   support

           0       0.98      0.89      0.93       819
           1       0.43      0.85      0.57        81

   micro avg       0.88      0.88      0.88       900
   macro avg       0.71      0.87      0.75       900
weighted avg       0.93      0.88      0.90       900

88.44444444444444


In [120]:
gnb = GaussianNB()
gnb.fit(X_train, Y_train)
y_pred = gnb.predict(X_test)
print confusion_matrix(Y_test,y_pred)
print classification_report(Y_test,y_pred)
print accuracy_score(Y_test, y_pred)*100

[[754  65]
 [ 37  44]]
              precision    recall  f1-score   support

           0       0.95      0.92      0.94       819
           1       0.40      0.54      0.46        81

   micro avg       0.89      0.89      0.89       900
   macro avg       0.68      0.73      0.70       900
weighted avg       0.90      0.89      0.89       900

88.66666666666667
